In [1]:
from dtgraph import Neo4jGraph, Rule, Transformation
hostname = "localhost"
password = ""
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="neo4j", username="", password=password)

In [2]:
from dtgraph.scenarios.movies import Movies
Movies.load(graph)

Flushed database: Deleted 296 nodes, deleted 1021 relationships, completed after 17 ms.
Load scenario: Added 171 labels, created 171 nodes, set 564 properties, created 253 relationships, completed after 773 ms.


### Writing the transformation

In [3]:
generate_actors = Rule('''
MATCH (n:Person)-[:ACTED_IN]->(:Movie)
GENERATE 
(x = (n):Actor { name = n.name, born = n.born })
''')
generate_directors = Rule('''
MATCH (n:Person)-[:DIRECTED]->(:Movie)
GENERATE 
(x=(n):Director { name = n.name, born = n.born})
''')
my_transform = Transformation([generate_actors, generate_directors])
my_transform.apply_on(graph)

Index: Added 1 index, completed after 5 ms.
Rule: Added 204 labels, created 102 nodes, set 446 properties, created 0 relationships, completed after 146 ms.
Rule: Added 51 labels, created 23 nodes, set 111 properties, created 0 relationships, completed after 69 ms.


215

### Adding a new rule to an existing transformation

In [4]:
generate_colleague = Rule('''
MATCH (n:Person)-[:DIRECTED]->(m:Movie)<-[:DIRECTED]-(o:Person)
GENERATE
(x = (n):)-[():COLLEAGUE { movie = m.title }]->(y = (o):)
''')
my_transform.add(generate_colleague)

Rule: Added 0 labels, created 0 nodes, set 28 properties, created 6 relationships, completed after 106 ms.


### Dealing with conflicts

In [5]:
my_transform.diagnose()

NodeConflicts: There are currently 0 nodes in the database which have a conflict.
EdgeConflicts: There are currently 2 edges in the database which have a conflict.
  (:Director {born: 1967, name: Lilly Wachowski})-[:COLLEAGUE {}]->(:Director {born: 1965, name: Lana Wachowski}) has a conflict on attributes ['movie']
  (:Director {born: 1965, name: Lana Wachowski})-[:COLLEAGUE {}]->(:Director {born: 1967, name: Lilly Wachowski}) has a conflict on attributes ['movie']


(0, 2)

In [6]:
my_transform.abort()
generate_colleague_fixed = Rule('''
MATCH (n:Person)-[:ACTED_IN]->(m:Movie)<-[:ACTED_IN]-(o:Person)
GENERATE
(x = (n):)-[(m):COLLEAGUE { movie = m.title }]->(y = (o):)
''')
my_transform_fixed = Transformation([generate_actors, generate_directors, generate_colleague_fixed])
my_transform_fixed.apply_on(graph)

Index: Removed 1 index, completed after 2 ms.
Abort: Deleted 125 nodes, deleted 6 relationships, completed after 16 ms.
Index: Added 1 index, completed after 3 ms.
Rule: Added 204 labels, created 102 nodes, set 446 properties, created 0 relationships, completed after 130 ms.
Rule: Added 51 labels, created 23 nodes, set 111 properties, created 0 relationships, completed after 67 ms.
Rule: Added 0 labels, created 0 nodes, set 1536 properties, created 768 relationships, completed after 125 ms.


322

In [7]:
my_transform_fixed.diagnose()

NodeConflicts: There are currently 0 nodes in the database which have a conflict.
EdgeConflicts: There are currently 0 edges in the database which have a conflict.


(0, 0)

### Validating the transformation

In [8]:
my_transform_fixed.eject()

Index: Removed 1 index, completed after 1 ms.
Eject: Removed 125 labels, erased 893 properties, completed after 396 ms.
